# 1. BIM to GIS Conversion

This Notebook imports revit files from BIM models and convert them to GIS geodatabase in prepartion for monitoring

Workflow:
1. Define Projection to PRS92
2. Export Feature to file geodatabase
3. Merge all feature layers
4. Delete unwanted fields
5. Add Fields
6. Calculate Field for Type
7. Calculate Field for SubType
8. Delete Fields

In [1]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"
workspace = r'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'
#workspace = r'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'
arcpy.env.workspace = workspace

In [2]:
## USE DEIFINITION
# First, makre sure to Add to Current Map all the models:
## e.g. 'StructuralColumns', 'Roofs'

# Name of Depot Building
dName = "FPH1"
fDataset = "Depot_Building"

path = os.path.join(workspace, fDataset)

In [3]:
# 1. Define Projection to PRS92
## 1.1. Add revit files to the Contents panel
## 1.2. Remove empty layers
## 1.3. Drag only layers of interest out of revit model and into Contents panel
## Note that 'Floors' in the BIM Model includes both slab and floors together.

## Architectural
a_column = r'Columns'
a_floor = r'Floors'
a_roof = r'Roofs'
a_wall = r'Walls'
a_ramp = r'Ramps'
a_stairRailing = r'StairsRailing'
a_stairs = r'Stairs'
a_site = r'Site'

## Structural
sframing = r'StructuralFraming'
sfoundation = r'StructuralFoundation'
scolumn = r'StructuralColumns'

## Define for each Depot Building
in_features = [a_floor, a_roof, a_stairs, a_wall,
           sframing, sfoundation, scolumn]

## Pick up only avaialble 
##in_features = [f for f in useLayer if f in categories]
##print(in_features)

sr = arcpy.SpatialReference("PRS 1992 Philippines Zone III")
for fl in in_features:
    arcpy.DefineProjection_management(fl, sr)
        

In [4]:
# 2. Export to file geodatabase
arcpy.FeatureClassToGeodatabase_conversion(in_features, path)


<Result 'C:\\Users\\eiji.LAPTOP-KSD9P6CP\\OneDrive - Oriental Consultants Global JV\\Documents\\ArcGIS\\Projects\\During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\Depot_Building'>

In [5]:
# 3. Merge all feature layers
arcpy.env.workspace = path
output_feature = "N2_DEP_" + dName + "_merged"
arcpy.Merge_management(in_features, output_feature,"","")

<Result 'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb\\Depot_Building\\N2_DEP_FPH1_merged'>

In [6]:
# 4. Delete Fields
## 4.1 Obtain fields to be kept
fieldList = [f.name for f in arcpy.ListFields(output_feature)]

# find OBJECTID, as this is not deletable
reg = re.compile(r'OBJECTID*|Shape|Shape_Length|Shape_Area|Shape.STArea()|Shape.STLength()|GlobalID')
objField = list(filter(reg.match, fieldList))
keepFields = objField + ['Category', 'Family', 'FamilyType','ObjectID', 'ObjectId', 'Bldg_Name', 'BldgLevel', 'OmniClassDescription', 'BldgLevel_Desc', 'DocName', 'Discipline']

# Drop fields
dropFields = [e for e in fieldList if e not in keepFields]

## 4.2 Delete fields from feature layer
arcpy.DeleteField_management(output_feature, dropFields)

<Result 'N2_DEP_FPH1_merged'>

In [7]:
# 5. Add Fields
addFields = ['Name', 'Type', 'SubType', 'Status', 'start_date', 'end_date', \
             'StructureType', 'StructureLevel', 'P6ID', 'Comment', \
            'GridX', 'GridY', 'CP', 'ID'] # ID is for bored piles

for field in addFields:
    if field in ('start_date', 'end_date'):
        arcpy.AddField_management(output_feature, field, "DATE", "", "", "", field, "NULLABLE", "")
    elif field in ('P6ID', 'Comment', 'GridX', 'GridY', 'CP', 'Name', 'ID'):
        arcpy.AddField_management(output_feature, field, "TEXT", "", "", "", field, "NULLABLE", "")
    else:
        arcpy.AddField_management(output_feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [8]:
# Set the initial status = 'To be Constructed'
with arcpy.da.UpdateCursor(output_feature, ['Status']) as cursor:
    for row in cursor:
        row[0] = 1
        cursor.updateRow(row)

In [9]:
with arcpy.da.UpdateCursor(output_feature, ['CP']) as cursor:
    for row in cursor:
        row[0] = 'N-05'
        cursor.updateRow(row)

In [10]:
# Assign Type based on Category field
### Domain Type (corresponding Category):
## 1. Structural Foundations (Structural Foundations)
## 2. Structural Columns (Structural Columns)
## 3. Structural Framing (Structural Framing)
## 4. Roofs (Roofs)
## 5. Floors (Floors)
## 6. Walls (Walls)
## 7. Columns (Columns)
## 8. Others

## note that 4 to 8 fall under Architectural disciplines of Structural revit file
with arcpy.da.UpdateCursor(output_feature, ['Category', 'Type']) as cursor:
    for row in cursor:
        if row[0] == 'Structural Foundations':
            row[1] = 1
        elif row[0] == 'Structural Columns':
            row[1] = 2
        elif row[0] == 'Structural Framing':
            row[1] = 3
        elif row[0] == 'Roofs':
            row[1] = 4
        elif row[0] == 'Floors':
            row[1] = 5
        elif row[0] == 'Walls':
            row[1] = 6
        elif row[0] == 'Columns':
            row[1] = 7
        else:
            row[1] = 8
        cursor.updateRow(row)

In [11]:
# Enter Depot building name
with arcpy.da.UpdateCursor(output_feature, ['Name']) as cursor:
    for row in cursor:
        if dName == "DEP":
            row[0] = "DEP"
        elif dName == "OCC":
            row[0] = "OCC"
        elif dName == "WS":
            row[0] = "WS"
        elif dName == "LRS":
            row[0] = "LRS"
        elif dName == "UCS":
            row[0] = "UCS"
        elif dName == "URS":
            row[0] = "URS"
        elif dName == "WRS":
            row[0] = "WRS"
        elif dName == "CMV":
            row[0] = "CMV"
        elif dName == "SCS":
            row[0] = "SCS"
        elif dName == "FPH1":
            row[0] = "FPH1"
        elif dName == "WOS":
            row[0] = "WOS"
        elif dName == "LOS":
            row[0] = "LOS"
        elif dName == "DHS":
            row[0] = "DHS"
        elif dName == "LGS":
            row[0] = "LGS"
        elif dName == "WGS":
            row[0] = "WGS"
        elif dName == "TGB":
            row[0] = "TGB"
        elif dName == "TMO":
            row[0] = "TMO"
        elif dName == "MCS":
            row[0] = "MCS"
        elif dName == "DBS1":
            row[0] = "DBS1"
        elif dName == "DBS2":
            row[0] = "DBS2"
        elif dName == "DB1":
            row[0] = "DB1"
        elif dName == "DB2":
            row[0] = "DB2"
        elif dName == "DSS":
            row[0] = "DSS"
        elif dName == "WPH1":
            row[0] = "WPH1"
        elif dName == "WPH2":
            row[0] = "WPH2"
        elif dName == "SH1":
            row[0] = "SH1"
        elif dName == "SH2":
            row[0] = "SH2"
        elif dName == "BPS":
            row[0] = "BPS"
        elif dName == "MPS":
            row[0] = "MPS"
        elif dName == "CPS":
            row[0] = "CPS"
        elif dName == "CER":
            row[0] = "CER"
        elif dName == "CNT":
            row[0] = "CNT"
        elif dName == "TRC":
            row[0] = "TRC"
        elif dName == "TRCE":
            row[0] = "TRCE"
        cursor.updateRow(row)


In [15]:
# Assign Domains
# 0. Domain Name
domainList = ['Station Structures_TYPE', 'Station Structures_SUBTYPE',\
              'Station Structures_STATUS', 'Station Structures_STRUCTURETYPE',\
              'Station Structures_STRUCTURELEVEL','N2_Depot_Building_NAME']

#domains = arcpy.da.ListDomains(workspace)
#listDomain = [d.name for d in domains]
#reg = re.compile(r'Station Structures*|Station*')
#listFinal = list(filter(reg.match, listDomain))

arcpy.AssignDomainToField_management(output_feature, 'Type', domainList[0])
arcpy.AssignDomainToField_management(output_feature, 'SubType', domainList[1])
arcpy.AssignDomainToField_management(output_feature, 'Status', domainList[2])
arcpy.AssignDomainToField_management(output_feature, 'StructureType', domainList[3])
arcpy.AssignDomainToField_management(output_feature, 'StructureLevel', domainList[4])
arcpy.AssignDomainToField_management(output_feature, 'Name', domainList[5])

<Result 'N2_DEP_FPH1_merged'>

In [1]:
# When you want to sort attribute table
temp_feature ="N2_Depot_Build_portal_copy_20230809"
out_feature = temp_feature + "_sorted"
sort_fields = [["Name","ASCENDING"], ["Type","ASCENDING"], ["ObjectId","ASCENDING"]]
arcpy.Sort_management(temp_feature, out_feature, sort_fields)

<Result 'C:\\Users\\eiji.LAPTOP-KSD9P6CP\\OneDrive - Oriental Consultants Global JV\\Documents\\ArcGIS\\Projects\\During-Construction_nscrexn2\\During-Construction_nscrexn2.gdb\\N2_Depot_Build_portal_copy_20230809_sorted'>

In [18]:
# Apply Symbology
symbolLyrx = r'C:/Users/emasu/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structures.lyrx'
arcpy.ApplySymbologyFromLayer_management(output_feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

# Update Existing Multipatch using New Layer

This code below edit existing N2 Depot Building multipatch layer (compiled one) using new subset
For exmple, when BIM revit on Workshop is updated, you can run this code to update the existing.

Workflow:
1. Convert BIM revit (new one) to feature dataset
2. Select and copy subset from existing multipatch layer
3. Pre-process the converted multipatch layer
    3.1. Add and delete fields in the converted multipatch layer (new one)
4. Join the copied subset from the existing multipatch to the pre-processed multipatch (step 3)
5. Truncate rows to be replaced with new ones in the existing multipatch
6. Append the new converted multipatch to the existing multipatch

In [ ]:
# 1. Convert BIM revit (new one) to feature dataset

In [ ]:
## Run steps in '1. BIM to GIS Conversion'

# Data Prep for Pile and Pile Cap of N2 Depot Buildings
   1. Bernie processes pile and pile caps from CAD and convert to individual multipatch layers
   2. I process these two layers for appending to N2_Depot_Bulding compiled database

In [1]:
# 0. 
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True

In [2]:
workspace = r'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'

In [ ]:
# 1. Compile pile and pile cap multipatch layers provided by Bernie

In [16]:
## 1.1. Define building name
bName = "URS"
CP = "N-05"

In [17]:
## 1.2. Define layers
pileL = bName + "_PILE"
pileCapL = bName + "_PILECAP"

In [21]:
## 1.3. Add important field names needed for appending
fields = ["Type", "SubType", "Name", "CP", "Status","Discipline","Category"]
for field in fields:
    if field in ["CP", "Name", "Category","Discipline"]:
        arcpy.AddField_management(pileL, field, "TEXT", "", "", "", field, "NULLABLE", "")
        arcpy.AddField_management(pileCapL, field, "TEXT", "", "", "", field, "NULLABLE", "")
        
    else:
        arcpy.AddField_management(pileL, field, "SHORT", "", "", "", field, "NULLABLE", "")
        arcpy.AddField_management(pileCapL, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [14]:
## 1.4. Add information
### Pile
with arcpy.da.UpdateCursor(pileL, fields) as cursor:
    for row in cursor:
        row[0] = 1
        row[1] = 1
        row[2] = bName
        row[3] = CP
        cursor.updateRow(row)
        
### Pile caps
with arcpy.da.UpdateCursor(pileCapL, fields) as cursor:
    for row in cursor:
        row[0] = 1
        row[1] = 2
        row[2] = bName
        row[3] = CP
        cursor.updateRow(row)

In [20]:
## 1.5. Append pile to pile Cap
arcpy.Append_management(pileL, pileCapL, schema_type = 'NO_TEST')

<Result 'URS_PILECAP'>

In [16]:
# 2. Clearn the compiled table
## 2.1. Convert string 'status' to numeric status
with arcpy.da.UpdateCursor(pileCapL, ["Status", "Status1"]) as cursor:
    for row in cursor:
        if row[0] == 'Completed':
            row[1] = 4
        else:
            row[1] = 1
        cursor.updateRow(row)

In [17]:
# 2.2. Delete old Status and alter field names
droppedField = 'Status'
listFields = [f.name for f in arcpy.ListFields(pileCapL)]
statusField = [f for f in listFields if f in droppedField]

if len(statusField) > 0:
    arcpy.DeleteField_management(pileCapL, droppedField)
    arcpy.AlterField_management(pileCapL, 'Status1', 'Status', 'Status')
else:
    print("Fied name 'Status' does not exist.")


In [19]:
# Alter the following field name to 'ID'
arcpy.AlterField_management(pileL, 'BoredPile_ID_Number', 'ID', 'ID')

<Result 'OCC_PileCap'>

In [20]:
# 3. Finally, append it to target layer
## 3.1. check they share the same coordinate system
targetLayer = 'N2_Depot_Building_portal'

sourceSP = arcpy.Describe(pileCapL).spatialReference.Name
targetSP = arcpy.Describe(targetLayer).spatialReference.Name

if sourceSP == targetSP:
    arcpy.Append_management(pileCapL, targetLayer, schema_type = 'NO_TEST')
else:
    print("WRONG! Different Coordinate System. Please check the layers.")


In [ ]:
# 4. When you delete or add observations to the target layer, you need to re-number 'uniqueID'
